<a href="https://colab.research.google.com/github/RealMyeong/AIFFELTHON/blob/main/JinMyeong/%EC%9D%BC%EB%B0%98%EB%8C%80%ED%99%94_%EC%B6%94%EA%B0%80_%ED%8C%8C%EC%9D%B4%ED%94%84%EB%9D%BC%EC%9D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 진행 순서

1. 데이터 불러오기
  - 일반 대화 데이터
  - 공격 대화 데이터
2. 키워드 추출
  - 공격 대화 데이터 키워드 추출
  - 일반 대화 데이터 키워드 추출
    - 화자 구분 x
    - 상담사 제외 고객만 키워드 추출
3. 키워드 비율에 따라 일반대화 추가

### 1. 데이터 불러오기

In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
# 일반 대화 데이터
normal_conv = pd.read_csv('/content/drive/MyDrive/AIFFELTHON/TUNiB/sns_data.csv')

# 공격 대화 데이터
attack_conv = pd.read_csv('/content/drive/MyDrive/AIFFELTHON/TUNiB/train_normalization.csv')

In [ ]:
attack_conv

,class,label_sentence,non_label_sentence,binary_class
0,직장 내 괴롭힘 대화,1:길동 씨 이번에 이것 좀 처리해요\n2:이거 제가 한 게 아닌데요\n1:팀에서 ...,길동 씨 이번에 이것 좀 처리해요\n이거 제가 한 게 아닌데요\n팀에서 내가 네가 ...,공격 대화
1,기타 괴롭힘 대화,1:야 너 앞니 왜 그렇게 튀어나왔냐\n2:태어날 때부터 그랬어 물어보지 마\n1:...,야 너 앞니 왜 그렇게 튀어나왔냐\n태어날 때부터 그랬어 물어보지 마\n아 그럼 태...,공격 대화
2,갈취 대화,1:원후야 학원 교재 사야 되는데 8만 원만 주면 안 돼\n2:미안 나 오늘은 진짜...,원후야 학원 교재 사야 되는데 8만 원만 주면 안 돼\n미안 나 오늘은 진짜 돈이 ...,공격 대화
3,갈취 대화,1:너 저번에 술 먹은 날 기억해\n2:아니 왜\n1:야 300만 원만 가져와\n2...,너 저번에 술 먹은 날 기억해\n아니 왜\n야 300만 원만 가져와\n뭔 소리야 ...,공격 대화
4,협박 대화,1:너 또 내말 무시하냐\n1:이 새끼 널 좆으로 보나 본데\n2:아냐 진짜 시간이...,너 또 내말 무시하냐\n이 새끼 널 좆으로 보나 본데\n아냐 진짜 시간이 없었어 미...,공격 대화
...,...,...,...,...
3865,기타 괴롭힘 대화,1:준하야 넌 대가리가 왜 이렇게 크냐\n2:내 머리가 뭐\n1:밥 먹으면 대가리만...,준하야 넌 대가리가 왜 이렇게 크냐\n내 머리가 뭐\n밥 먹으면 대가리만 크냐 너는...,공격 대화
3866,갈취 대화,1:내가 지금 너 아들 김길준 데리고 있어 살리고 싶으면 계좌에 1억만 보내\n2...,내가 지금 너 아들 김길준 데리고 있어 살리고 싶으면 계좌에 1억만 보내\n예 ...,공격 대화
3867,직장 내 괴롭힘 대화,1:나는 씨 같은 사람 보면 참 신기하더라 어떻게 저렇게 살지\n1:왜 그래 들...,나는 씨 같은 사람 보면 참 신기하더라 어떻게 저렇게 살지\n왜 그래 들리겠어\...,공격 대화
3868,갈취 대화,1:누구 맘대로 여기서 장사하래\n2:이게 무슨 일입니까\n1:남의 구역에서 장사하...,누구 맘대로 여기서 장사하래\n이게 무슨 일입니까\n남의 구역에서 장사하려면 자릿세...,공격 대화


In [ ]:
# attack_conv.to_csv('/content/drive/MyDrive/AIFFELTHON/TUNiB/train_normalization.csv', index=False)

In [ ]:
normal_conv

,label_sentence,non_label_sentence,class,binary_class,type
0,1:야 니 남친은 잘 지내고 있니\n2:이야 한숨만 나온다\n2:엊그제 그래도 40...,야 니 남친은 잘 지내고 있니\n이야 한숨만 나온다\n엊그제 그래도 400일이었어\...,일반 대화,일반 대화,SNS
1,1:나 궁금한 게 하나 있어\n2:응 그게 뭔데 자기야\n1:작은 아빠는 풍산개를 ...,나 궁금한 게 하나 있어\n응 그게 뭔데 자기야\n작은 아빠는 풍산개를 어디서 그렇...,일반 대화,일반 대화,SNS
2,1:너 최근에 했던 운동이 뭐야\n2:나 등산 갔었지\n2:관악산 올라가 봤나\n1...,너 최근에 했던 운동이 뭐야\n나 등산 갔었지\n관악산 올라가 봤나\n관악산이 어디...,일반 대화,일반 대화,SNS
3,1:니가 최근에 했던 게임이 뭐였지\n2:나 오버 워치 너무 사랑하지\n1:요즘 애...,니가 최근에 했던 게임이 뭐였지\n나 오버 워치 너무 사랑하지\n요즘 애들은 총 쏘...,일반 대화,일반 대화,SNS
4,1:여행 대선배로서 추천할 여행지 있니\n2:나 최근에 부산에 다녀왔잖아 좋더라\n...,여행 대선배로서 추천할 여행지 있니\n나 최근에 부산에 다녀왔잖아 좋더라\n하 나 ...,일반 대화,일반 대화,SNS
...,...,...,...,...,...
26993,1:나 진짜 너무 피곤하다\n2:너 또 과제 하느라 밤 샜지\n1:우리 과는 시험 ...,나 진짜 너무 피곤하다\n너 또 과제 하느라 밤 샜지\n우리 과는 시험 대신 과제를...,일반 대화,일반 대화,SNS
26994,1:다들 요즘에 뭐 공부하고 있는 것 있어\n2:나는 특별히 공부하는 것은 없어\n...,다들 요즘에 뭐 공부하고 있는 것 있어\n나는 특별히 공부하는 것은 없어\n졸업시험...,일반 대화,일반 대화,SNS
26995,1:요즘 헬스장 다닌다며 잘 다니고 있어\n2:응 하루에 1 2시간 하고 있어\n1...,요즘 헬스장 다닌다며 잘 다니고 있어\n응 하루에 1 2시간 하고 있어\n유산소 위...,일반 대화,일반 대화,SNS
26996,1:요즘 날이 더운데 대중교통에 사람들도 많아서 너무 힘든 거 같아요\n2:네 너무...,요즘 날이 더운데 대중교통에 사람들도 많아서 너무 힘든 거 같아요\n네 너무 힘들어...,일반 대화,일반 대화,SNS


In [ ]:
import numpy as np
idx = []
batch_idx = []

for i in range(len(normal_conv)):
  batch_idx.append(i)
  if len(batch_idx)==10:
    idx.append(batch_idx)
    batch_idx = []

In [ ]:
len(idx)

2699

In [ ]:
batch_sen = []
for i in idx:
  a = normal_conv.iloc[i].reset_index(drop=True).non_label_sentence
  sen = []
  for j in a:
    t = j.split('\n')
    sen += t

  batch_sen.append(sen)

In [ ]:
batch_sen[0]

In [ ]:
len(batch_sen)

2699

### 2. 키워드 추출

1. 공격 대화 키워드 추출


In [ ]:
!pip install krwordrank

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def keyword(class_name, data, n=20):
    genre = data[data['class']==class_name].reset_index(drop=True).label_sentence

    attack, damaged = [], []

    for e in genre:
        e = e.split('\n')
        for e_ in e:
            if e_[0] == '1':
                attack.append(e_[2:])
            else:
                damaged.append(e_[2:])

    from krwordrank.sentence import summarize_with_sentences

    texts = attack
    keywords_att, sents = summarize_with_sentences(texts, num_keywords=100, 
                                           num_keysents=10, max_length=10, diversity=0.5)

    texts = damaged
    keywords_dam, sents = summarize_with_sentences(texts, num_keywords=100, 
                                           num_keysents=10, max_length=10, diversity=0.5)

    print('{}의 가해자 :'.format(class_name),keywords_att.keys())
    print()
    print('{}의 피해자 :'.format(class_name),keywords_dam.keys())
    
    keywords_att = list(keywords_att.keys())
    keywords_dam = list(keywords_dam.keys())
    
    target_attack, target_damaged = [], []

    for i in range(100):
        if (keywords_att[i] not in keywords_dam) : target_attack.append(keywords_att[i])
        if len(target_attack) == n: break 

    for i in range(100):
        if (keywords_dam[i] not in keywords_att) : target_damaged.append(keywords_dam[i])
        if len(target_damaged) == n: break
    
    print()
    print('{}의 가해자(중복제거) :'.format(class_name),target_attack)
    print()
    print('{}의 피해자(중복제거) :'.format(class_name),target_damaged)

    return [target_attack, target_damaged]

In [ ]:
extort_att, extort_vic = keyword('갈취 대화', attack_conv)

갈취 대화의 가해자 : dict_keys(['내가', '아니', '그럼', '그래', '빌려', '지금', '그냥', '내놔', '이거', '네가', '원만', '빨리', '진짜', '우리', '친구', '돈이', '거야', '오늘', '없어', '있는', '그러', '가져', '나도', '나오면', '말고', '전에', '제가', '있어', '여기', '사람', '보내', '좋은', '10', '너네', '알아', '너무', '이번', '거기', '새끼', '엄마', '어떻게', '그거', '얼마', '당장', '이렇게', '그렇게', '내일', '뒤져서', '말이', '나한테', '무슨', '어이', '당신', '줄래', '좋아', '그런', '가지고', '어디', '하나', '싫으면', '생각', '돈을', '하는', '다른', '거지', '테니', '그건', '가방', '없는', '근데', '이게', '지갑', '많이', '주면', '누가', '이런', '얼른', '줘봐', '돌려', '내놓', '하면', '맞고', '바로', '한번', '줄게', '학교', '하고', '저기', '입금', '신고', '되는', '필요', '잠깐', '사진', '정도', '그게', '다시', '이제', '와봐', '조용히'])

갈취 대화의 피해자 : dict_keys(['아니', '제발', '진짜', '없어', '안돼', '무슨', '이거', '내가', '돈이', '정말', '미안', '그래', '제가', '그러', '거야', '알겠', '그건', '여기', '어떻게', '지금', '죄송', '엄마', '빌려', '돌려', '그런', '이번', '나도', '그럼', '없는', '이게', '이러', '저희', '그거', '이건', '있어', '그게', '저번', '거예요', '우리', '말이', '너무', '마세요', '가져', '오늘', '주세요', '그렇', '빨리', '얼마', '드릴', '어디', '이제', '근데', '번만', '그냥', '누구', '받은', '

In [ ]:
work_att, work_vic = keyword('직장 내 괴롭힘 대화', attack_conv)

직장 내 괴롭힘 대화의 가해자 : dict_keys(['내가', '아니', '거야', '그래', '지금', '오늘', '자네', '그럼', '우리', '그렇게', '하는', '사람', '그냥', '이렇게', '무슨', '네가', '김대리', '이번', '이거', '하고', '일을', '진짜', '그러', '그런', '말이', '없어', '있어', '어떻게', '대리', '말고', '요즘', '너무', '이런', '하나', '일이', '내일', '다른', '생각', '죄송', '알아', '제대로', '보고', '휴가', '아주', '있는', '빨리', '누가', '못하', '하면', '같은', '어디', '다시', '해야', '가서', '상사', '업무', '하지', '커피', '여기', '건가', '들어', '부탁', '그게', '같이', '뭐가', '싶어', '거지', '이게', '없는', '씨가', '시간', '새끼', '씨는', '근데', '못해', '남자', '와서', '퇴근', '나한테', '일도', '나가', '것도', '아직', '어제', '있나', '얼마나', '이제', '하라', '앞으로', '회식', '생활', '과장', '다니', '똑바로', '너는', '머리', '부장님', '얘기', '혼자', '자기'])

직장 내 괴롭힘 대화의 피해자 : dict_keys(['제가', '죄송', '아니', '부장님', '무슨', '아닙니다', '너무', '과장님', '정말', '다시', '말씀', '지금', '일이', '그건', '저도', '하겠습니다', '그래도', '그런', '오늘', '팀장님', '알겠', '그게', '저는', '이건', '대리님', '업무', '있어서', '열심히', '사장님', '같습니', '그러', '아닌', '빨리', '있습니다', '다른', '그럼', '그렇', '내일', '진짜', '휴가', '없습니다', '같아', '이번', '잘못', '앞으로', '하고', '어떻게', '있는', '바로', '아직', '아뇨', '혹

In [ ]:
threat_att, threat_vic = keyword('협박 대화', attack_conv)

협박 대화의 가해자 : dict_keys(['내가', '거야', '네가', '지금', '아니', '진짜', '그래', '그냥', '그럼', '죽여', '우리', '사람', '있어', '알아', '하나', '그러', '죽어', '어떻게', '전에', '당장', '죽고', '새끼', '어디', '정신', '빨리', '당신', '없어', '하면', '너도', '그렇게', '너네', '무슨', '나도', '이렇게', '여기', '싶어', '가족', '있는', '오늘', '이게', '나한테', '생각', '이제', '아주', '헤어', '같이', '말이', '들어', '하는', '않으면', '그런', '죽이', '칼로', '테니', '나랑', '장기', '소리', '말고', '잘못', '이거', '죽는', '때문', '정말', '했지', '죽을', '대로', '너희', '보고', '죽일', '자꾸', '너는', '하지', '다시', '뭐가', '집에', '수가', '같은', '이런', '신고', '얼마나', '아직', '하고', '거니까', '한번', '모르', '좋은', '몰라', '조용히', '싫으면', '식으로', '다른', '가져', '가만', '계속', '없는', '싶지', '말하', '너를', '여자', '목숨'])

협박 대화의 피해자 : dict_keys(['제발', '아니', '무슨', '정말', '내가', '제가', '우리', '거야', '죄송', '살려', '그러', '진짜', '지금', '그래', '잘못', '어떻게', '네가', '미안', '너무', '그런', '이러', '사람', '주세요', '그만', '그렇게', '그냥', '경찰', '알겠', '말아', '그건', '마세요', '그게', '이제', '거예요', '시간', '신고', '없어', '협박', '소리', '이거', '조금만', '나는', '당신', '아닙니다', '선생님', '하지', '하는', '여기', '그럼', '저는', '건들지', '나도', '생각', '다시', '진정', 

In [ ]:
other_att, other_vic = keyword('기타 괴롭힘 대화', attack_conv)

기타 괴롭힘 대화의 가해자 : dict_keys(['아니', '내가', '네가', '진짜', '그냥', '그래', '우리', '거야', '그럼', '여기', '지금', '너무', '무슨', '그렇게', '사람', '그러', '새끼', '이렇게', '어디', '빨리', '이거', '어떻게', '그런', '너네', '하는', '얼굴', '친구', '장애', '냄새', '말고', '거지', '같은', '뭐가', '엄마', '돼지', '다니', '있어', '것도', '너는', '없어', '못생', '하지', '알아', '기분', '하고', '이게', '정말', '이런', '나는', '남자', '오늘', '다른', '너도', '집에', '말이', '애들', '건데', '그만', '부모', '같이', '누가', '학교', '있는', '환불', '다시', '하나', '당장', '없는', '저기', '근데', '생각', '뭐야', '싫어', '찐따', '사장', '나도', '그거', '너랑', '소리', '제가', '장난', '따라', '하면', '에이', '좋아', '보고', '나오', '존나', '들어', '이제', '다리', '얘들아', '머리', '그게', '다녀', '너한테', '같아', '공부', '주제에', '얼마나'])

기타 괴롭힘 대화의 피해자 : dict_keys(['아니', '무슨', '내가', '손님', '그만', '제발', '거야', '제가', '죄송', '그래', '그런', '너무', '진짜', '그러', '미안', '지금', '그렇', '말이', '우리', '하지', '그냥', '나도', '그게', '정말', '네가', '있어', '저희', '하는', '그건', '여기', '엄마', '사람', '어떻게', '이거', '나한', '기분', '잘못', '그럼', '이러', '선생님', '다른', '말씀', '다시', '자꾸', '환불', '혹시', '하고', '오늘', '너네', '얘들아', '일이', '마세요', '나는', '이런', '말을', '

위 키워드를 바탕으로 중복을 제거한 가해자들의 키워드를 따로 저장해줌.

피해자는 대부분 `네? 죄송합니다. 미안 ` 등의 키워드가 나와서 비슷한 모습을 볼 수 있음.

공격적인 내용의 대부분은 가해자가 하는 말에의해 결정된다고 볼 수 있음.

In [ ]:
att_keyword = extort_att + work_att + threat_att + other_att

In [ ]:
# 중복 제거
att_keyword = list(set(att_keyword))
len(att_keyword)

66

In [ ]:
batch_sen[1]

['나 당근 마켓에 화장품 올렸던 거 알아',
 '당근 마켓이 뭐야',
 '그런 건 처음 듣는데  마트야',
 '너 어린 애가 당근 마켓도 모르니',
 '그러게 나 문찐인가 봐 키키',
 '뭐 하는 건데',
 '동네 사람들이랑 거래하는 거야',
 '아  택배 거래 아니고 직거래',
 '어 근데 택배 거래도 상대방이랑 얘기해서 가능해',
 '오 괜찮다 키키',
 '그렇지만 무섭지 않아',
 '돈 들고 튀면 어떡해',
 '그래서 되도록 현금 가져오라고 하지',
 '오    계좌 이체 하는 건 아니고',
 '응 그것도 해도 되고 나는 되도록 현금 추구함',
 '그렇구나 화장품 뭐 팔았어',
 '나 몰래 내 꺼 판 거 아니지',
 '그럴 리가  올영에서 샀는데 양이 많아서 하나 올렸지',
 '그런 거 있으면 나나 주지 왜 팔아',
 '아니  너랑 사는 곳이 멀잖아',
 '그건 인정 당근 마켓 유용하네 키키',
 '나 몰랐는데 박보검 군대 갔니',
 '어휴 간 지가 언젠데  우리 오빠 열일 하잖아',
 '맞아 해군으로 갔다고 하더라',
 '그래 사진 봤어',
 '정복 잘 어울려 보검씨',
 '야 사람들이 박보검 잘생겼다고 했잖아',
 '이제 알았잖아 그거 보고 키키',
 '나 무슨 군대 드라마 한편 본 줄 알았어 키키',
 '진짜 잘생겼더라',
 '나는 한 장밖에 못 봤는데 여러 장 있니',
 '공유 좀',
 '네이버 기사에 많이 올라 오더라',
 '확실히 박보검 최고',
 '보통 연예인들 군대 사진 보면 일반 사람 같던데 박보검은 진짜 연예인이더라 키키',
 '어 맞아 군대 사진으로 옥택연도 유명한데 봤어',
 '옥택연은 못 봤는데 그 사람도 해군이야',
 '아니 육군으로 간 것 같더라',
 '피지컬이 더 좋아졌어',
 '육군이면 이끼룩 입었겠네 전역했어',
 '전역한 것 같던데  요즘 티비에 자주 봐',
 '아 진짜  나는 요즘에 티비를 안 봐서 몰랐네',
 '티비도 좀 보고 살아 언니',
 '너의 최애 영화는 뭐야',
 '나 작년에 나오 테넷이 내 최애 영화야',

In [ ]:
def select_normal_conversation(sen, att_keyword, threshold=0.6):

  
  from krwordrank.sentence import summarize_with_sentences

  texts = sen
  keywords_client, sents = summarize_with_sentences(texts, num_keywords=20, 
                                           num_keysents=5, max_length=10, diversity=0.5)

  keywords_client = list(keywords_client.keys())
  keywords_att = att_keyword

  length = len(keywords_client)

  clean_keywords = []
  for word in keywords_client:
    if not word in keywords_att:
      clean_keywords.append(word)
  
  clean_ratio = len(clean_keywords) / length
  if clean_ratio > threshold:
    return True
  else:
    return False

In [ ]:
select_batch_idx = []

for i in range(len(batch_sen)):
  try:
    if select_normal_conversation(batch_sen[i], att_keyword):
      select_batch_idx.append(i)
  except:
    continue

In [ ]:
len(select_batch_idx)

2615

In [ ]:
normal_idx = []
for i in select_batch_idx:
  normal_idx += idx[i]

In [ ]:
len(normal_idx)

26150

In [ ]:
filtered_normal_conv = normal_conv.iloc[normal_idx]
filtered_normal_conv

,label_sentence,non_label_sentence,class,binary_class,type
0,1:야 니 남친은 잘 지내고 있니\n2:이야 한숨만 나온다\n2:엊그제 그래도 40...,야 니 남친은 잘 지내고 있니\n이야 한숨만 나온다\n엊그제 그래도 400일이었어\...,일반 대화,일반 대화,SNS
1,1:나 궁금한 게 하나 있어\n2:응 그게 뭔데 자기야\n1:작은 아빠는 풍산개를 ...,나 궁금한 게 하나 있어\n응 그게 뭔데 자기야\n작은 아빠는 풍산개를 어디서 그렇...,일반 대화,일반 대화,SNS
2,1:너 최근에 했던 운동이 뭐야\n2:나 등산 갔었지\n2:관악산 올라가 봤나\n1...,너 최근에 했던 운동이 뭐야\n나 등산 갔었지\n관악산 올라가 봤나\n관악산이 어디...,일반 대화,일반 대화,SNS
3,1:니가 최근에 했던 게임이 뭐였지\n2:나 오버 워치 너무 사랑하지\n1:요즘 애...,니가 최근에 했던 게임이 뭐였지\n나 오버 워치 너무 사랑하지\n요즘 애들은 총 쏘...,일반 대화,일반 대화,SNS
4,1:여행 대선배로서 추천할 여행지 있니\n2:나 최근에 부산에 다녀왔잖아 좋더라\n...,여행 대선배로서 추천할 여행지 있니\n나 최근에 부산에 다녀왔잖아 좋더라\n하 나 ...,일반 대화,일반 대화,SNS
...,...,...,...,...,...
26985,1:전공때문에 학원을 다녀야 할 것 같아\n2:어떤 학원에 다녀야 하는데\n1:응 ...,전공때문에 학원을 다녀야 할 것 같아\n어떤 학원에 다녀야 하는데\n응 컴퓨터 학...,일반 대화,일반 대화,SNS
26986,1:배구 경기 보러 가고 싶다\n2:나도 직관 가고 싶어\n1:코로나 좀 풀...,배구 경기 보러 가고 싶다\n나도 직관 가고 싶어\n코로나 좀 풀려서 이제 ...,일반 대화,일반 대화,SNS
26987,1:내일 발표해야하는데 너무 떨린다\n2:너라면 발표 잘 할 수 있어\n1:그건 아...,내일 발표해야하는데 너무 떨린다\n너라면 발표 잘 할 수 있어\n그건 아니고 개인발...,일반 대화,일반 대화,SNS
26988,1:너네 저번에 도쿄 올림픽 경기들 생각나\n2:너무 인상깊은 경기들이 많았어서 당...,너네 저번에 도쿄 올림픽 경기들 생각나\n너무 인상깊은 경기들이 많았어서 당연히 아...,일반 대화,일반 대화,SNS


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/AIFFELTHON/TUNiB/real_norm_data.csv')
df

,type,non_label_sentence,label_sentence,class,binary_class
0,고객센터,네 침구 추가를 해야 할 것 같아서요\n네 1세트가 필요하신 건가요\n네 맞습니다 ...,1:반갑습니다 입니다\n2:네 침구 추가를 해야 할 것 같아서요\n1:네 1세트가 ...,일반 대화,일반 대화
1,고객센터,네 예약하려면 어떻게 해야 하나요\n네 회원 가입 후 로그인 하셔서 예약하시면 되세...,1:반갑습니다 입니다\n2:네 예약하려면 어떻게 해야 하나요\n1:네 회원 가입 후...,일반 대화,일반 대화
2,고객센터,네 객실 인원이 추가될 경우엔 어떻게 하면 되나요\n아 일반적인 스탠다드 디럭스 객...,1:반갑습니다 입니다\n2:네 객실 인원이 추가될 경우엔 어떻게 하면 되나요\n1:...,일반 대화,일반 대화
3,고객센터,네 듣고 보니 코인 세탁 방이 있다고 하던 데요\n아 코인 런더리룸이요\n네 어디에...,1:반갑습니다 입니다\n2:네 듣고 보니 코인 세탁 방이 있다고 하던 데요\n1:아...,일반 대화,일반 대화
4,고객센터,네 혹시 부대 시설로 편의점이 있나요\n네 호텔 1층에 위치해 있습니다\n아 어떤 ...,1:반갑습니다 입니다\n2:네 혹시 부대 시설로 편의점이 있나요\n1:네 호텔 1층...,일반 대화,일반 대화
...,...,...,...,...,...
106225,카카오,일본 진짜 가고 싶다\n나 요새 맨날 이 애기 하잖아\n인정 솔직히 너 일본에서 명...,1:일본 진짜 가고 싶다\n1:나 요새 맨날 이 애기 하잖아\n2:인정 솔직히 너 ...,일반 대화,일반 대화
106226,카카오,오늘 친구네 강아지 산책하는데 따라갔다 왔다\n와 부러워 전에 그 바닷가 사는 애\...,1:오늘 친구네 강아지 산책하는데 따라갔다 왔다\n2:와 부러워 전에 그 바닷가 사...,일반 대화,일반 대화
106227,카카오,요즘에 엄마가 나 씻고 화장하면 남자 만나냐고 물어봄\n갑자기 집이랑 만 왔다 갔다...,1:요즘에 엄마가 나 씻고 화장하면 남자 만나냐고 물어봄\n2:갑자기 집이랑 만 왔...,일반 대화,일반 대화
106228,카카오,주말에 보드 타러 갈래 사상에 괜찮은 데 있음 잘 가르치던대\n뭐 바퀴 달린 거 그...,1:주말에 보드 타러 갈래 사상에 괜찮은 데 있음 잘 가르치던대\n2:뭐 바퀴 달린...,일반 대화,일반 대화


In [ ]:
norm_data = pd.concat([filtered_normal_conv, df], ignore_index=True)
norm_data

,label_sentence,non_label_sentence,class,binary_class,type
0,1:야 니 남친은 잘 지내고 있니\n2:이야 한숨만 나온다\n2:엊그제 그래도 40...,야 니 남친은 잘 지내고 있니\n이야 한숨만 나온다\n엊그제 그래도 400일이었어\...,일반 대화,일반 대화,SNS
1,1:나 궁금한 게 하나 있어\n2:응 그게 뭔데 자기야\n1:작은 아빠는 풍산개를 ...,나 궁금한 게 하나 있어\n응 그게 뭔데 자기야\n작은 아빠는 풍산개를 어디서 그렇...,일반 대화,일반 대화,SNS
2,1:너 최근에 했던 운동이 뭐야\n2:나 등산 갔었지\n2:관악산 올라가 봤나\n1...,너 최근에 했던 운동이 뭐야\n나 등산 갔었지\n관악산 올라가 봤나\n관악산이 어디...,일반 대화,일반 대화,SNS
3,1:니가 최근에 했던 게임이 뭐였지\n2:나 오버 워치 너무 사랑하지\n1:요즘 애...,니가 최근에 했던 게임이 뭐였지\n나 오버 워치 너무 사랑하지\n요즘 애들은 총 쏘...,일반 대화,일반 대화,SNS
4,1:여행 대선배로서 추천할 여행지 있니\n2:나 최근에 부산에 다녀왔잖아 좋더라\n...,여행 대선배로서 추천할 여행지 있니\n나 최근에 부산에 다녀왔잖아 좋더라\n하 나 ...,일반 대화,일반 대화,SNS
...,...,...,...,...,...
132375,1:일본 진짜 가고 싶다\n1:나 요새 맨날 이 애기 하잖아\n2:인정 솔직히 너 ...,일본 진짜 가고 싶다\n나 요새 맨날 이 애기 하잖아\n인정 솔직히 너 일본에서 명...,일반 대화,일반 대화,카카오
132376,1:오늘 친구네 강아지 산책하는데 따라갔다 왔다\n2:와 부러워 전에 그 바닷가 사...,오늘 친구네 강아지 산책하는데 따라갔다 왔다\n와 부러워 전에 그 바닷가 사는 애\...,일반 대화,일반 대화,카카오
132377,1:요즘에 엄마가 나 씻고 화장하면 남자 만나냐고 물어봄\n2:갑자기 집이랑 만 왔...,요즘에 엄마가 나 씻고 화장하면 남자 만나냐고 물어봄\n갑자기 집이랑 만 왔다 갔다...,일반 대화,일반 대화,카카오
132378,1:주말에 보드 타러 갈래 사상에 괜찮은 데 있음 잘 가르치던대\n2:뭐 바퀴 달린...,주말에 보드 타러 갈래 사상에 괜찮은 데 있음 잘 가르치던대\n뭐 바퀴 달린 거 그...,일반 대화,일반 대화,카카오


In [ ]:
norm_data.to_csv('/content/drive/MyDrive/AIFFELTHON/TUNiB/real_sns_data.csv', index=False)

### 3. 비율에 따라 일반대화 추가

먼저 카카오톡 대화와 고객센터 대화의 비율은 3:1로 정했습니다.

기존의 공격대화에서 볼 때 직장 내 괴롭힘 클래스에 해당하는 대화를 제외하고는 대부분 가벼운 말투(야, 너, 가져와, 해라 등등)입니다.

추가할 일반 데이터인 카카오톡 데이터와 고객센터 데이터를 비교해보면 카카오톡 데이터가 비교적 가벼운 말투로 이루어져있고, 고객센터 대화가 조금 더 딱딱한 말투로 이루어져 있기 때문에 이에 맞춰서 데이터의 비율을 정했습니다.

먼저 공격대화를 일반 대화와 비슷하게 한글 숫자 제외하고 전부 전처리 해줍니다.

In [ ]:
import pandas as pd
import numpy as np
import re

# 일반 대화 데이터
# normal_conv = pd.read_csv('/content/drive/MyDrive/AIFFELTHON/TUNiB/real_sns_data.csv')

# 공격 대화 데이터
attack_conv = pd.read_csv('/content/drive/MyDrive/AIFFELTHON/TUNiB/Augmentation/ri_50.csv')

In [ ]:
normal_conv

,label_sentence,non_label_sentence,class,binary_class,type
0,1:야 니 남친은 잘 지내고 있니\n2:이야 한숨만 나온다\n2:엊그제 그래도 40...,야 니 남친은 잘 지내고 있니\n이야 한숨만 나온다\n엊그제 그래도 400일이었어\...,일반 대화,일반 대화,SNS
1,1:나 궁금한 게 하나 있어\n2:응 그게 뭔데 자기야\n1:작은 아빠는 풍산개를 ...,나 궁금한 게 하나 있어\n응 그게 뭔데 자기야\n작은 아빠는 풍산개를 어디서 그렇...,일반 대화,일반 대화,SNS
2,1:너 최근에 했던 운동이 뭐야\n2:나 등산 갔었지\n2:관악산 올라가 봤나\n1...,너 최근에 했던 운동이 뭐야\n나 등산 갔었지\n관악산 올라가 봤나\n관악산이 어디...,일반 대화,일반 대화,SNS
3,1:니가 최근에 했던 게임이 뭐였지\n2:나 오버 워치 너무 사랑하지\n1:요즘 애...,니가 최근에 했던 게임이 뭐였지\n나 오버 워치 너무 사랑하지\n요즘 애들은 총 쏘...,일반 대화,일반 대화,SNS
4,1:여행 대선배로서 추천할 여행지 있니\n2:나 최근에 부산에 다녀왔잖아 좋더라\n...,여행 대선배로서 추천할 여행지 있니\n나 최근에 부산에 다녀왔잖아 좋더라\n하 나 ...,일반 대화,일반 대화,SNS
...,...,...,...,...,...
132375,1:일본 진짜 가고 싶다\n1:나 요새 맨날 이 애기 하잖아\n2:인정 솔직히 너 ...,일본 진짜 가고 싶다\n나 요새 맨날 이 애기 하잖아\n인정 솔직히 너 일본에서 명...,일반 대화,일반 대화,카카오
132376,1:오늘 친구네 강아지 산책하는데 따라갔다 왔다\n2:와 부러워 전에 그 바닷가 사...,오늘 친구네 강아지 산책하는데 따라갔다 왔다\n와 부러워 전에 그 바닷가 사는 애\...,일반 대화,일반 대화,카카오
132377,1:요즘에 엄마가 나 씻고 화장하면 남자 만나냐고 물어봄\n2:갑자기 집이랑 만 왔...,요즘에 엄마가 나 씻고 화장하면 남자 만나냐고 물어봄\n갑자기 집이랑 만 왔다 갔다...,일반 대화,일반 대화,카카오
132378,1:주말에 보드 타러 갈래 사상에 괜찮은 데 있음 잘 가르치던대\n2:뭐 바퀴 달린...,주말에 보드 타러 갈래 사상에 괜찮은 데 있음 잘 가르치던대\n뭐 바퀴 달린 거 그...,일반 대화,일반 대화,카카오


In [ ]:
attack_conv

,class,label_sentence,non_label_sentence,binary_class
0,갈취 대화,1:노랑아 나 담 주 생일인 거 알지\n2:네 아 네 미리 축하드려요\n1:뭐야 ...,노랑 파랑 아 나 나의 담 좇 주 주기도 생일 즉위식 인 거 것 알 울 지\n 네 ...,공격 대화
1,협박 대화,2:우리 사랑해서 헤어진 거 아니었어 왜 지금 와서 다시 나타나는 건데\n1:당신...,우리 그대 사랑 슬픔 해서 헤어진 거 것 것 아니었어 왜 과연 지금 현재 와서 다시...,공격 대화
2,직장 내 괴롭힘 대화,2:일 더 있나요 이제 퇴근시간도 한참 지났는데요\n1:가긴 어딜 가 더 해야지\...,일 월과 더 던 있나요 이제 그때 퇴근 출퇴근 시간 분간 도 한참 며칠 지났는데요\...,공격 대화
3,기타 괴롭힘 대화,1:야 이 새끼 봐봐 오타쿠 인가 봐\n1:뭔데 뭔데 아 뭐냐 그림 이게 그 미미...,야 이 그러 새끼 암컷 봐 봐 오타쿠 대중문화 인가 봐\n 뭔 데 뭔 데 아 뭐 ....,공격 대화
4,직장 내 괴롭힘 대화,1:지금 바쁜데 무슨 휴가가 안된다\n2:부장님 저희 딸이 지금 많이 아픕니다 ...,지금 현재 바쁜데 무슨 어찌 휴가 잔치 가 안된다\n 부장 본부장 님 저희 그분 딸...,공격 대화
...,...,...,...,...
5865,기타 괴롭힘 대화,1:준하야 넌 대가리가 왜 이렇게 크냐\n2:내 머리가 뭐\n1:밥 먹으면 대가리만...,준하야 넌 대가리가 왜 이렇게 크냐\n내 머리가 뭐\n밥 먹으면 대가리만 크냐 너는...,공격 대화
5866,갈취 대화,1:내가 지금 너 아들 김길준 데리고 있어 살리고 싶으면 계좌에 1억만 보내\n2...,내가 지금 너 아들 김길준 데리고 있어 살리고 싶으면 계좌에 1억만 보내\n예 ...,공격 대화
5867,직장 내 괴롭힘 대화,1:나는 씨 같은 사람 보면 참 신기하더라 어떻게 저렇게 살지\n1:왜 그래 들...,나는 씨 같은 사람 보면 참 신기하더라 어떻게 저렇게 살지\n왜 그래 들리겠어\...,공격 대화
5868,갈취 대화,1:누구 맘대로 여기서 장사하래\n2:이게 무슨 일입니까\n1:남의 구역에서 장사하...,누구 맘대로 여기서 장사하래\n이게 무슨 일입니까\n남의 구역에서 장사하려면 자릿세...,공격 대화


In [ ]:
def preprocess(sen):
  sen = sen.split('\n')
  
  clean_sen = []
  for i in sen:
    i = i.strip()
    i = re.sub(r'[^0-9가-힣:]', ' ', i)
    i = i.strip()
    clean_sen.append(i)

  return '\n'.join(clean_sen)

In [ ]:
normal_conv['label_sentence'] = normal_conv['label_sentence'].apply(lambda x : preprocess(x))
normal_conv

,label_sentence,non_label_sentence,class,binary_class,type
0,1:야 니 남친은 잘 지내고 있니\n2:이야 한숨만 나온다\n2:엊그제 그래도 40...,야 니 남친은 잘 지내고 있니\n이야 한숨만 나온다\n엊그제 그래도 400일이었어\...,일반 대화,일반 대화,SNS
1,1:나 궁금한 게 하나 있어\n2:응 그게 뭔데 자기야\n1:작은 아빠는 풍산개를 ...,나 궁금한 게 하나 있어\n응 그게 뭔데 자기야\n작은 아빠는 풍산개를 어디서 그렇...,일반 대화,일반 대화,SNS
2,1:너 최근에 했던 운동이 뭐야\n2:나 등산 갔었지\n2:관악산 올라가 봤나\n1...,너 최근에 했던 운동이 뭐야\n나 등산 갔었지\n관악산 올라가 봤나\n관악산이 어디...,일반 대화,일반 대화,SNS
3,1:니가 최근에 했던 게임이 뭐였지\n2:나 오버 워치 너무 사랑하지\n1:요즘 애...,니가 최근에 했던 게임이 뭐였지\n나 오버 워치 너무 사랑하지\n요즘 애들은 총 쏘...,일반 대화,일반 대화,SNS
4,1:여행 대선배로서 추천할 여행지 있니\n2:나 최근에 부산에 다녀왔잖아 좋더라\n...,여행 대선배로서 추천할 여행지 있니\n나 최근에 부산에 다녀왔잖아 좋더라\n하 나 ...,일반 대화,일반 대화,SNS
...,...,...,...,...,...
132375,1:일본 진짜 가고 싶다\n1:나 요새 맨날 이 애기 하잖아\n2:인정 솔직히 너 ...,일본 진짜 가고 싶다\n나 요새 맨날 이 애기 하잖아\n인정 솔직히 너 일본에서 명...,일반 대화,일반 대화,카카오
132376,1:오늘 친구네 강아지 산책하는데 따라갔다 왔다\n2:와 부러워 전에 그 바닷가 사...,오늘 친구네 강아지 산책하는데 따라갔다 왔다\n와 부러워 전에 그 바닷가 사는 애\...,일반 대화,일반 대화,카카오
132377,1:요즘에 엄마가 나 씻고 화장하면 남자 만나냐고 물어봄\n2:갑자기 집이랑 만 왔...,요즘에 엄마가 나 씻고 화장하면 남자 만나냐고 물어봄\n갑자기 집이랑 만 왔다 갔다...,일반 대화,일반 대화,카카오
132378,1:주말에 보드 타러 갈래 사상에 괜찮은 데 있음 잘 가르치던대\n2:뭐 바퀴 달린...,주말에 보드 타러 갈래 사상에 괜찮은 데 있음 잘 가르치던대\n뭐 바퀴 달린 거 그...,일반 대화,일반 대화,카카오


In [ ]:
normal_conv['non_label_sentence'] = normal_conv['non_label_sentence'].apply(lambda x : preprocess(x))
normal_conv

,label_sentence,non_label_sentence,class,binary_class,type
0,1:야 니 남친은 잘 지내고 있니\n2:이야 한숨만 나온다\n2:엊그제 그래도 40...,야 니 남친은 잘 지내고 있니\n이야 한숨만 나온다\n엊그제 그래도 400일이었어\...,일반 대화,일반 대화,SNS
1,1:나 궁금한 게 하나 있어\n2:응 그게 뭔데 자기야\n1:작은 아빠는 풍산개를 ...,나 궁금한 게 하나 있어\n응 그게 뭔데 자기야\n작은 아빠는 풍산개를 어디서 그렇...,일반 대화,일반 대화,SNS
2,1:너 최근에 했던 운동이 뭐야\n2:나 등산 갔었지\n2:관악산 올라가 봤나\n1...,너 최근에 했던 운동이 뭐야\n나 등산 갔었지\n관악산 올라가 봤나\n관악산이 어디...,일반 대화,일반 대화,SNS
3,1:니가 최근에 했던 게임이 뭐였지\n2:나 오버 워치 너무 사랑하지\n1:요즘 애...,니가 최근에 했던 게임이 뭐였지\n나 오버 워치 너무 사랑하지\n요즘 애들은 총 쏘...,일반 대화,일반 대화,SNS
4,1:여행 대선배로서 추천할 여행지 있니\n2:나 최근에 부산에 다녀왔잖아 좋더라\n...,여행 대선배로서 추천할 여행지 있니\n나 최근에 부산에 다녀왔잖아 좋더라\n하 나 ...,일반 대화,일반 대화,SNS
...,...,...,...,...,...
132375,1:일본 진짜 가고 싶다\n1:나 요새 맨날 이 애기 하잖아\n2:인정 솔직히 너 ...,일본 진짜 가고 싶다\n나 요새 맨날 이 애기 하잖아\n인정 솔직히 너 일본에서 명...,일반 대화,일반 대화,카카오
132376,1:오늘 친구네 강아지 산책하는데 따라갔다 왔다\n2:와 부러워 전에 그 바닷가 사...,오늘 친구네 강아지 산책하는데 따라갔다 왔다\n와 부러워 전에 그 바닷가 사는 애\...,일반 대화,일반 대화,카카오
132377,1:요즘에 엄마가 나 씻고 화장하면 남자 만나냐고 물어봄\n2:갑자기 집이랑 만 왔...,요즘에 엄마가 나 씻고 화장하면 남자 만나냐고 물어봄\n갑자기 집이랑 만 왔다 갔다...,일반 대화,일반 대화,카카오
132378,1:주말에 보드 타러 갈래 사상에 괜찮은 데 있음 잘 가르치던대\n2:뭐 바퀴 달린...,주말에 보드 타러 갈래 사상에 괜찮은 데 있음 잘 가르치던대\n뭐 바퀴 달린 거 그...,일반 대화,일반 대화,카카오


In [ ]:
attack_conv['non_label_sentence'] = attack_conv['non_label_sentence'].apply(lambda x : preprocess(x))
attack_conv

,class,label_sentence,non_label_sentence,binary_class
0,갈취 대화,1:노랑아 나 담 주 생일인 거 알지\n2:네 아 네 미리 축하드려요\n1:뭐야 ...,노랑 파랑 아 나 나의 담 좇 주 주기도 생일 즉위식 인 거 것 알 울 지\n네 카...,공격 대화
1,협박 대화,2:우리 사랑해서 헤어진 거 아니었어 왜 지금 와서 다시 나타나는 건데\n1:당신...,우리 그대 사랑 슬픔 해서 헤어진 거 것 것 아니었어 왜 과연 지금 현재 와서 다시...,공격 대화
2,직장 내 괴롭힘 대화,2:일 더 있나요 이제 퇴근시간도 한참 지났는데요\n1:가긴 어딜 가 더 해야지\...,일 월과 더 던 있나요 이제 그때 퇴근 출퇴근 시간 분간 도 한참 며칠 지났는데요\...,공격 대화
3,기타 괴롭힘 대화,1:야 이 새끼 봐봐 오타쿠 인가 봐\n1:뭔데 뭔데 아 뭐냐 그림 이게 그 미미...,야 이 그러 새끼 암컷 봐 봐 오타쿠 대중문화 인가 봐\n뭔 데 뭔 데 아 뭐 ...,공격 대화
4,직장 내 괴롭힘 대화,1:지금 바쁜데 무슨 휴가가 안된다\n2:부장님 저희 딸이 지금 많이 아픕니다 ...,지금 현재 바쁜데 무슨 어찌 휴가 잔치 가 안된다\n부장 본부장 님 저희 그분 딸 ...,공격 대화
...,...,...,...,...
5865,기타 괴롭힘 대화,1:준하야 넌 대가리가 왜 이렇게 크냐\n2:내 머리가 뭐\n1:밥 먹으면 대가리만...,준하야 넌 대가리가 왜 이렇게 크냐\n내 머리가 뭐\n밥 먹으면 대가리만 크냐 너는...,공격 대화
5866,갈취 대화,1:내가 지금 너 아들 김길준 데리고 있어 살리고 싶으면 계좌에 1억만 보내\n2...,내가 지금 너 아들 김길준 데리고 있어 살리고 싶으면 계좌에 1억만 보내\n예 ...,공격 대화
5867,직장 내 괴롭힘 대화,1:나는 씨 같은 사람 보면 참 신기하더라 어떻게 저렇게 살지\n1:왜 그래 들...,나는 씨 같은 사람 보면 참 신기하더라 어떻게 저렇게 살지\n왜 그래 들리겠어\...,공격 대화
5868,갈취 대화,1:누구 맘대로 여기서 장사하래\n2:이게 무슨 일입니까\n1:남의 구역에서 장사하...,누구 맘대로 여기서 장사하래\n이게 무슨 일입니까\n남의 구역에서 장사하려면 자릿세...,공격 대화


1. 일반 대화 1000개 추가 (카카오톡 750 : 고객센터 250)

In [ ]:
norm_2000 = normal_conv.sample(1600)
norm_2000

,label_sentence,non_label_sentence,class,binary_class,type
91072,1:반갑습니다 해피 상담센터 입니다\n1:어떤 문의가 있으십니까\n2:네 지금 식당...,해피 상담센터\n어떤 문의가 있으십니까\n네 지금 식당을 하는데 융자 신청 좀 하려...,일반 대화,일반 대화,고객센터
65117,1:여기 뻘은 살아있네\n2:꽃게랑 조개가 엄청 많아\n1:애들이 신나서 잘노네\n...,여기 뻘은 살아있네\n꽃게랑 조개가 엄청 많아\n애들이 신나서 잘노네\n오들 꽃게 ...,일반 대화,일반 대화,카카오
54701,1:조기 유학은 아이들에게 좋은 선택일까\n2:그건 정말 복불복이라고 봐\n1:더러...,조기 유학은 아이들에게 좋은 선택일까\n그건 정말 복불복이라고 봐\n더러는 혼자 떠...,일반 대화,일반 대화,카카오
14620,1:햄스터 같이 키울 사람\n2:난 고슴도치 키울래\n1:아니다\n1:소라게 키울래...,햄스터 같이 키울 사람\n난 고슴도치 키울래\n아니다\n소라게 키울래\n햄스터는 징...,일반 대화,일반 대화,SNS
98518,1: 입니다 무엇을 도와 드릴까요\n2: 에 마이크로화이바솜이 들어갔다는데 이게 뭐...,도와 드릴까요\n에 마이크로화이바솜이 들어갔다는데 이게 뭐에요\n마이크로원사로 이루...,일반 대화,일반 대화,고객센터
...,...,...,...,...,...
53455,1:오빠 주토피아 영화관에서 봤어\n2:음 기억이 잘 안난다\n1:아 그래 그 주토...,오빠 주토피아 영화관에서 봤어\n음 기억이 잘 안난다\n아 그래 그 주토피아 나는...,일반 대화,일반 대화,카카오
62439,1:지하철이 너무 지연되는데\n2:그러게 말야 사고 난건가\n1:방송이라도 해줘야 ...,지하철이 너무 지연되는데\n그러게 말야 사고 난건가\n방송이라도 해줘야 되는거아냐\...,일반 대화,일반 대화,카카오
33897,1:기분 좋은 하루시작 상담원 입니다\n2:네 안녕하세요 궁금한 게 있어서요\n1:...,기분 좋은 하루시작\n네 궁금한 게 있어서요\n네 편하게 말씀해주세요\n이번에 ...,일반 대화,일반 대화,고객센터
43130,1:요즘 지구온난화로 일교차가 크네요\n2:맞아 일교차도 심하고 봄 가을이 삭제되는...,요즘 지구온난화로 일교차가 크네요\n맞아 일교차도 심하고 봄 가을이 삭제되는 것...,일반 대화,일반 대화,카카오


In [ ]:
train_rs_2000 = pd.concat([norm_2000.iloc[:, :-1], attack_conv], ignore_index=True)
train_rs_2000

,label_sentence,non_label_sentence,class,binary_class
0,1:반갑습니다 해피 상담센터 입니다\n1:어떤 문의가 있으십니까\n2:네 지금 식당...,해피 상담센터\n어떤 문의가 있으십니까\n네 지금 식당을 하는데 융자 신청 좀 하려...,일반 대화,일반 대화
1,1:여기 뻘은 살아있네\n2:꽃게랑 조개가 엄청 많아\n1:애들이 신나서 잘노네\n...,여기 뻘은 살아있네\n꽃게랑 조개가 엄청 많아\n애들이 신나서 잘노네\n오들 꽃게 ...,일반 대화,일반 대화
2,1:조기 유학은 아이들에게 좋은 선택일까\n2:그건 정말 복불복이라고 봐\n1:더러...,조기 유학은 아이들에게 좋은 선택일까\n그건 정말 복불복이라고 봐\n더러는 혼자 떠...,일반 대화,일반 대화
3,1:햄스터 같이 키울 사람\n2:난 고슴도치 키울래\n1:아니다\n1:소라게 키울래...,햄스터 같이 키울 사람\n난 고슴도치 키울래\n아니다\n소라게 키울래\n햄스터는 징...,일반 대화,일반 대화
4,1: 입니다 무엇을 도와 드릴까요\n2: 에 마이크로화이바솜이 들어갔다는데 이게 뭐...,도와 드릴까요\n에 마이크로화이바솜이 들어갔다는데 이게 뭐에요\n마이크로원사로 이루...,일반 대화,일반 대화
...,...,...,...,...
7465,1:준하야 넌 대가리가 왜 이렇게 크냐\n2:내 머리가 뭐\n1:밥 먹으면 대가리만...,준하야 넌 대가리가 왜 이렇게 크냐\n내 머리가 뭐\n밥 먹으면 대가리만 크냐 너는...,기타 괴롭힘 대화,공격 대화
7466,1:내가 지금 너 아들 김길준 데리고 있어 살리고 싶으면 계좌에 1억만 보내\n2...,내가 지금 너 아들 김길준 데리고 있어 살리고 싶으면 계좌에 1억만 보내\n예 ...,갈취 대화,공격 대화
7467,1:나는 씨 같은 사람 보면 참 신기하더라 어떻게 저렇게 살지\n1:왜 그래 들...,나는 씨 같은 사람 보면 참 신기하더라 어떻게 저렇게 살지\n왜 그래 들리겠어\...,직장 내 괴롭힘 대화,공격 대화
7468,1:누구 맘대로 여기서 장사하래\n2:이게 무슨 일입니까\n1:남의 구역에서 장사하...,누구 맘대로 여기서 장사하래\n이게 무슨 일입니까\n남의 구역에서 장사하려면 자릿세...,갈취 대화,공격 대화


In [ ]:
train_rs_2000.to_csv('/content/drive/MyDrive/AIFFELTHON/TUNiB/experiment_data/train_RI50_1600.csv', index=False)

2. 일반 대화 4000개 추가 (카카오 3000 : 고객센터 1000)

In [ ]:
kakao_3000 = kakao_data.sample(3000)
kakao_3000

,type,non_label_sentence,label_sentence,class,binary_class
52577,카카오,연말 다가오는데 가족 모임계획 있어\n코로나때문에 올해도 안 할것 같아\n그런데 한...,1:연말 다가오는데 가족 모임계획 있어\n2:코로나때문에 올해도 안 할것 같아\n1...,일반 대화,일반 대화
15516,카카오,미어캣 본 적 있어\n동물원에서나 한 번씩 봤던 거 같아요\n미어캣 서있는 모습 귀...,1:미어캣 본 적 있어\n2:동물원에서나 한 번씩 봤던 거 같아요\n1:미어캣 서있...,일반 대화,일반 대화
35744,카카오,정월 대보름 아침에 깨물어 먹는 견과류에 땅콩이 들어가잖아\n그렇지 우리나라 전통...,1:정월 대보름 아침에 깨물어 먹는 견과류에 땅콩이 들어가잖아\n2:그렇지 우리나라...,일반 대화,일반 대화
37952,카카오,자기 눈 성형 해봤어\n아니 난 자연미인이야\n역시 우리 자기는 그랬구나\n눈 예뻐...,1:자기 눈 성형 해봤어\n2:아니 난 자연미인이야\n1:역시 우리 자기는 그랬구나...,일반 대화,일반 대화
12064,카카오,다음 주부터 개춥데\n맞아 이럴 때는 태국 가야 댐\n다들 한파 대비 단단히 하렴\...,1:다음 주부터 개춥데\n2:맞아 이럴 때는 태국 가야 댐\n1:다들 한파 대비 단...,일반 대화,일반 대화
...,...,...,...,...,...
57750,카카오,다들 혹시 평소에 생각해 둔 이상형 있어\n글쎄 그건 갑자기 왜 물어\n별 건 아...,1:다들 혹시 평소에 생각해 둔 이상형 있어\n2:글쎄 그건 갑자기 왜 물어\n1:...,일반 대화,일반 대화
42701,카카오,이마트 배달시켰어\n나도 배달 시켰는데\n이마트에 먹을게 많더라\n이마트 살거 무궁...,1:이마트 배달시켰어\n2:나도 배달 시켰는데\n1:이마트에 먹을게 많더라\n2:이...,일반 대화,일반 대화
59573,카카오,야구 동호회 지금도 하다니\n그런데 넌 왜 그만 뒀어\n그거 활동을 주말에 하잖아\...,1:야구 동호회 지금도 하다니\n2:그런데 넌 왜 그만 뒀어\n1:그거 활동을 주말...,일반 대화,일반 대화
35216,카카오,너 의경 나왔다며 의경은 무슨 일해\n의경은 의무경찰에 준 말이야\n경찰을 도와서...,1:너 의경 나왔다며 의경은 무슨 일해\n2:의경은 의무경찰에 준 말이야\n2:경찰...,일반 대화,일반 대화


In [ ]:
cs_1000 = cs_data.sample(1000)
cs_1000

,type,non_label_sentence,label_sentence,class,binary_class
78130,고객센터,궁금한 게 있어서 연락드립니다\n네\n제가 일이 있어서 집에 자주 못 가거든요\n혹...,1:안녕하세요 상담원 입니다\n2:안녕하세요 궁금한 게 있어서 연락드립니다\n1:네...,일반 대화,일반 대화
72311,고객센터,예 제가 좀 전에 전화드렸는데요 벽걸이 티비를요\n스탠드 형태로 바꾸고 싶다고 말씀...,1:반갑습니다 입니다\n2:예 제가 좀 전에 전화드렸는데요 벽걸이 티비를요\n2:스...,일반 대화,일반 대화
79509,고객센터,예 벨트를 하나 봤는데요\n이거를 구매를 좀 할까 해서요\n네 전화 주문 원하시는 ...,1: 상담원입니다 안녕하세요\n2:예 벨트를 하나 봤는데요\n2:이거를 구매를 좀 ...,일반 대화,일반 대화
61453,고객센터,전화 좀 몇 개 물어볼게요\n네 어떤 전화 가 필요하신가요\n이요\n선생님 로는 확...,1:반갑습니다 입니다 무엇을 도와드릴까요\n2:전화 좀 몇 개 물어볼게요\n1:네 ...,일반 대화,일반 대화
85663,고객센터,네 여보세요 궁금한 게 있는데요\n사회복지사 1급 과정이 마지막이라고 들었거든요\...,1: 입니다 무엇을 도와드릴까요\n2:네 여보세요 안녕하세요 궁금한 게 있는데요\n...,일반 대화,일반 대화
...,...,...,...,...,...
59,고객센터,네 최신식 시설로 편안히 모시는\n아 네 객실 사용에 대해 궁금한 게 있어서요\n네...,1:네 최신식 시설로 편안히 모시는 입니다\n2:아 네 객실 사용에 대해 궁금한 게...,일반 대화,일반 대화
89644,고객센터,전화 주셔서 고객센터\n저 문의할게 있어서요\n\n교재 주문했는데 이거 발송이 됐...,1: 전화 주셔서 감사합니다 고객센터 상담원 입니다\n2: 안녕하세요 저 문의할게 ...,일반 대화,일반 대화
65614,고객센터,네 다름이 아니라 제가 이번에 문어를 양식하다가요\n다른 어종으로 양식 그거를 변...,1:안녕하세요 입니다\n2:네 안녕하세요 다름이 아니라 제가 이번에 문어를 양식하다...,일반 대화,일반 대화
64526,고객센터,선생님 좀 도와주세요\n어떤 일 이신가요 선생님\n할머니가 다리를 다치셔서 못 움직...,1:반갑습니다 입니다\n2:선생님 좀 도와주세요\n1:어떤 일 이신가요 선생님\n2...,일반 대화,일반 대화


In [ ]:
norm_4000 = pd.concat([kakao_3000, cs_1000], ignore_index=True)
norm_4000

,type,non_label_sentence,label_sentence,class,binary_class
0,카카오,연말 다가오는데 가족 모임계획 있어\n코로나때문에 올해도 안 할것 같아\n그런데 한...,1:연말 다가오는데 가족 모임계획 있어\n2:코로나때문에 올해도 안 할것 같아\n1...,일반 대화,일반 대화
1,카카오,미어캣 본 적 있어\n동물원에서나 한 번씩 봤던 거 같아요\n미어캣 서있는 모습 귀...,1:미어캣 본 적 있어\n2:동물원에서나 한 번씩 봤던 거 같아요\n1:미어캣 서있...,일반 대화,일반 대화
2,카카오,정월 대보름 아침에 깨물어 먹는 견과류에 땅콩이 들어가잖아\n그렇지 우리나라 전통...,1:정월 대보름 아침에 깨물어 먹는 견과류에 땅콩이 들어가잖아\n2:그렇지 우리나라...,일반 대화,일반 대화
3,카카오,자기 눈 성형 해봤어\n아니 난 자연미인이야\n역시 우리 자기는 그랬구나\n눈 예뻐...,1:자기 눈 성형 해봤어\n2:아니 난 자연미인이야\n1:역시 우리 자기는 그랬구나...,일반 대화,일반 대화
4,카카오,다음 주부터 개춥데\n맞아 이럴 때는 태국 가야 댐\n다들 한파 대비 단단히 하렴\...,1:다음 주부터 개춥데\n2:맞아 이럴 때는 태국 가야 댐\n1:다들 한파 대비 단...,일반 대화,일반 대화
...,...,...,...,...,...
3995,고객센터,네 최신식 시설로 편안히 모시는\n아 네 객실 사용에 대해 궁금한 게 있어서요\n네...,1:네 최신식 시설로 편안히 모시는 입니다\n2:아 네 객실 사용에 대해 궁금한 게...,일반 대화,일반 대화
3996,고객센터,전화 주셔서 고객센터\n저 문의할게 있어서요\n\n교재 주문했는데 이거 발송이 됐...,1: 전화 주셔서 감사합니다 고객센터 상담원 입니다\n2: 안녕하세요 저 문의할게 ...,일반 대화,일반 대화
3997,고객센터,네 다름이 아니라 제가 이번에 문어를 양식하다가요\n다른 어종으로 양식 그거를 변...,1:안녕하세요 입니다\n2:네 안녕하세요 다름이 아니라 제가 이번에 문어를 양식하다...,일반 대화,일반 대화
3998,고객센터,선생님 좀 도와주세요\n어떤 일 이신가요 선생님\n할머니가 다리를 다치셔서 못 움직...,1:반갑습니다 입니다\n2:선생님 좀 도와주세요\n1:어떤 일 이신가요 선생님\n2...,일반 대화,일반 대화


In [ ]:
train_4000 = pd.concat([norm_4000.iloc[:,1:], attack_conv])
train_4000

,non_label_sentence,label_sentence,class,binary_class
0,연말 다가오는데 가족 모임계획 있어\n코로나때문에 올해도 안 할것 같아\n그런데 한...,1:연말 다가오는데 가족 모임계획 있어\n2:코로나때문에 올해도 안 할것 같아\n1...,일반 대화,일반 대화
1,미어캣 본 적 있어\n동물원에서나 한 번씩 봤던 거 같아요\n미어캣 서있는 모습 귀...,1:미어캣 본 적 있어\n2:동물원에서나 한 번씩 봤던 거 같아요\n1:미어캣 서있...,일반 대화,일반 대화
2,정월 대보름 아침에 깨물어 먹는 견과류에 땅콩이 들어가잖아\n그렇지 우리나라 전통...,1:정월 대보름 아침에 깨물어 먹는 견과류에 땅콩이 들어가잖아\n2:그렇지 우리나라...,일반 대화,일반 대화
3,자기 눈 성형 해봤어\n아니 난 자연미인이야\n역시 우리 자기는 그랬구나\n눈 예뻐...,1:자기 눈 성형 해봤어\n2:아니 난 자연미인이야\n1:역시 우리 자기는 그랬구나...,일반 대화,일반 대화
4,다음 주부터 개춥데\n맞아 이럴 때는 태국 가야 댐\n다들 한파 대비 단단히 하렴\...,1:다음 주부터 개춥데\n2:맞아 이럴 때는 태국 가야 댐\n1:다들 한파 대비 단...,일반 대화,일반 대화
...,...,...,...,...
7735,준하야 머리가 왜 이렇게 커\n내 머리는 무엇입니까\n밥을 먹으면 머리만 커지나요...,1: 준하야 머리가 왜 이렇게 커\n2: 내 머리는 무엇입니까\n1: 밥을 먹으면...,기타 괴롭힘 대화,공격 대화
7736,당신의 아들 김길준을 데리고 갑니다 그를 구하고 싶다면 그의 계좌로 1억을 보내라...,1: 당신의 아들 김길준을 데리고 갑니다 그를 구하고 싶다면 그의 계좌로 1억을 ...,갈취 대화,공격 대화
7737,선생님 같은 사람들을 보면 정말 이상해요 어떻게 이렇게 살 수 있습니까\n왜 그런...,1: 선생님 같은 사람들을 보면 정말 이상해요 어떻게 이렇게 살 수 있습니까\n1...,직장 내 괴롭힘 대화,공격 대화
7738,여기서 사업을 하고 싶은 사람\n뭐야\n남의 동네에서 장사를 하려면 집세를 내야 한...,1: 여기서 사업을 하고 싶은 사람\n2: 뭐야\n1: 남의 동네에서 장사를 하려면...,갈취 대화,공격 대화


In [ ]:
train_4000.to_csv('/content/drive/MyDrive/AIFFELTHON/TUNiB/experiment_data/train_bt_4000.csv', index=False)

3. 일반 대화 10000개 추가 (카카오 7500 : 고객센터 2500)

In [ ]:
kakao_7500 = kakao_data.sample(7500)
kakao_7500

,type,non_label_sentence,label_sentence,class,binary_class
19520,카카오,엄마 워치4 사준거 잘 사용해야하는데\n잘사용하시는데\n근데 반짝 사용하는걸수도\n...,1:엄마 워치4 사준거 잘 사용해야하는데\n2:잘사용하시는데\n1:근데 반짝 사용하...,일반 대화,일반 대화
13273,카카오,요새 스터디 그런거 많이 하던데\n스터디\n어떤 거 스터디해\n닌 관심 없니\n내는...,1:요새 스터디 그런거 많이 하던데\n2:스터디\n2:어떤 거 스터디해\n1: 닌 ...,일반 대화,일반 대화
107236,카카오,오늘 해장국 먹고 옴\n오 무슨 해장국 먹었어요\n홍천 뚝배기 앎 서남 시장에\n오...,1:오늘 해장국 먹고 옴\n2:오 무슨 해장국 먹었어요\n1: 홍천 뚝배기 앎 서남...,일반 대화,일반 대화
23475,카카오,마블 호크아이 영화 나오나봐\n자기 그 캐릭터 별로잖아\n그렇긴 한데 2세대 ...,1:마블 호크아이 영화 나오나봐\n2:자기 그 캐릭터 별로잖아\n1:그렇긴 한데 2...,일반 대화,일반 대화
35509,카카오,빕스가까\n확인했나 가\n어어 아직있더라\n죽이제 이번년도 까지인가봐\n...,1:빕스가까\n2:확인했나 가\n1:어어 아직있더라\n1:죽이제 이번년도 까지인가봐...,일반 대화,일반 대화
...,...,...,...,...,...
94967,카카오,다들 대중교통 이용 하니\n당연하지 지하철 완전 편함 하하\n지하철이나 버스 이용하...,1:다들 대중교통 이용 하니\n2:당연하지 지하철 완전 편함 하하\n1:지하철이나 ...,일반 대화,일반 대화
45652,카카오,오늘 개막했어 미국에서는 기사가 엄청 쏟아지고 있어\n드디어 2021 20...,1:오늘 개막했어 미국에서는 기사가 엄청 쏟아지고 있어\n2:드디어 2021 202...,일반 대화,일반 대화
91667,카카오,차사면 로 사서 차박 다니고 싶다\n재밌겠다 요즘 의자 접을 수 있는 차도 나왔더라...,1:차사면 로 사서 차박 다니고 싶다\n2:재밌겠다 요즘 의자 접을 수 있는 차도 ...,일반 대화,일반 대화
14437,카카오,운동 좋아하는 거 있음\n저 헬스 좋아합니다\n낸 운동은 다 좋아함\n은근 그냥 아...,1:운동 좋아하는 거 있음\n2:저 헬스 좋아합니다\n1:낸 운동은 다 좋아함\n2...,일반 대화,일반 대화


In [ ]:
cs_2500 = cs_data.sample(2500)
cs_2500

,type,non_label_sentence,label_sentence,class,binary_class
4234,고객센터,에 있는 오리배 타 보려고 하는데요\n이용 요금이 혹시 얼마인지 알 수 있을까요\n...,1:반갑습니다 상담원입니다\n2:안녕하세요 에 있는 오리배 타 보려고 하는데요\n2...,일반 대화,일반 대화
87163,고객센터,안녕하십니까\n지금 애들이 과학수업을 듣고 싶어 해서요\n혹시 과학수업은 실험 위주...,1: 안녕하십니까 상담원 입니다\n2: 지금 애들이 과학수업을 듣고 싶어 해서요\n...,일반 대화,일반 대화
90484,고객센터,좋은 아침\n수강 신청 등록 금액을 확인했는데 다음 화면으로 안 넘어가지네요\n껐다...,1: 좋은 아침입니다 상담원 입니다\n2: 수강 신청 등록 금액을 확인했는데 다음 ...,일반 대화,일반 대화
83757,고객센터,네 문의드릴 게 있어서요\n네 말씀주시면 안내 드리겠습니다\n장바구니에 담아논 이...,1:안녕하세요 상담원 입니다\n2:네 안녕하세요 문의드릴 게 있어서요\n1:네 말씀...,일반 대화,일반 대화
80247,고객센터,근데 저희 쪽에서 지금 전산상으로 봤을 때는요\n반품에 대해 아무런 처리가 안 돼 ...,1:근데 저희 쪽에서 지금 전산상으로 봤을 때는요\n1:반품에 대해 아무런 처리가 ...,일반 대화,일반 대화
...,...,...,...,...,...
69996,고객센터,다자녀가정에 대해서 좀 물어보려구요\n네 어떤 부분이 궁금하세요\n다자녀가정 기준이...,1: 입니다 무엇을 도와드릴까요\n2:다자녀가정에 대해서 좀 물어보려구요\n1:네 ...,일반 대화,일반 대화
75320,고객센터,전화주셔서\n\n을 사용하는데 버튼이 잘 눌리지 않아요\n터치 버튼 주위 물기가 있...,1:전화주셔서 감사합니다 입니다\n1: 무엇을 도와드릴까요\n2: 을 사용하는데 버...,일반 대화,일반 대화
66440,고객센터,문의할 게 있어서요\n네 어떤 문의이실까요\n개인사업자인데 공동사업 해지하려고 합니...,1:안녕하세요 상담원 입니다\n2:안녕하세요 문의할 게 있어서요\n1:네 어떤 문의...,일반 대화,일반 대화
81052,고객센터,배송 주문 이력 먼저 확인 후 안내해드리겠습니다\n주문자분 성함이 되십니까\n제 은...,1: 배송 주문 이력 먼저 확인 후 안내해드리겠습니다\n1:주문자분 성함이 되십니까...,일반 대화,일반 대화


In [ ]:
norm_10000 = pd.concat([kakao_7500, cs_2500], ignore_index=True)
norm_10000

,type,non_label_sentence,label_sentence,class,binary_class
0,카카오,엄마 워치4 사준거 잘 사용해야하는데\n잘사용하시는데\n근데 반짝 사용하는걸수도\n...,1:엄마 워치4 사준거 잘 사용해야하는데\n2:잘사용하시는데\n1:근데 반짝 사용하...,일반 대화,일반 대화
1,카카오,요새 스터디 그런거 많이 하던데\n스터디\n어떤 거 스터디해\n닌 관심 없니\n내는...,1:요새 스터디 그런거 많이 하던데\n2:스터디\n2:어떤 거 스터디해\n1: 닌 ...,일반 대화,일반 대화
2,카카오,오늘 해장국 먹고 옴\n오 무슨 해장국 먹었어요\n홍천 뚝배기 앎 서남 시장에\n오...,1:오늘 해장국 먹고 옴\n2:오 무슨 해장국 먹었어요\n1: 홍천 뚝배기 앎 서남...,일반 대화,일반 대화
3,카카오,마블 호크아이 영화 나오나봐\n자기 그 캐릭터 별로잖아\n그렇긴 한데 2세대 ...,1:마블 호크아이 영화 나오나봐\n2:자기 그 캐릭터 별로잖아\n1:그렇긴 한데 2...,일반 대화,일반 대화
4,카카오,빕스가까\n확인했나 가\n어어 아직있더라\n죽이제 이번년도 까지인가봐\n...,1:빕스가까\n2:확인했나 가\n1:어어 아직있더라\n1:죽이제 이번년도 까지인가봐...,일반 대화,일반 대화
...,...,...,...,...,...
9995,고객센터,다자녀가정에 대해서 좀 물어보려구요\n네 어떤 부분이 궁금하세요\n다자녀가정 기준이...,1: 입니다 무엇을 도와드릴까요\n2:다자녀가정에 대해서 좀 물어보려구요\n1:네 ...,일반 대화,일반 대화
9996,고객센터,전화주셔서\n\n을 사용하는데 버튼이 잘 눌리지 않아요\n터치 버튼 주위 물기가 있...,1:전화주셔서 감사합니다 입니다\n1: 무엇을 도와드릴까요\n2: 을 사용하는데 버...,일반 대화,일반 대화
9997,고객센터,문의할 게 있어서요\n네 어떤 문의이실까요\n개인사업자인데 공동사업 해지하려고 합니...,1:안녕하세요 상담원 입니다\n2:안녕하세요 문의할 게 있어서요\n1:네 어떤 문의...,일반 대화,일반 대화
9998,고객센터,배송 주문 이력 먼저 확인 후 안내해드리겠습니다\n주문자분 성함이 되십니까\n제 은...,1: 배송 주문 이력 먼저 확인 후 안내해드리겠습니다\n1:주문자분 성함이 되십니까...,일반 대화,일반 대화


In [ ]:
train_10000 = pd.concat([norm_10000.iloc[:, 1:], attack_conv], ignore_index=True)
train_10000

,non_label_sentence,label_sentence,class,binary_class
0,엄마 워치4 사준거 잘 사용해야하는데\n잘사용하시는데\n근데 반짝 사용하는걸수도\n...,1:엄마 워치4 사준거 잘 사용해야하는데\n2:잘사용하시는데\n1:근데 반짝 사용하...,일반 대화,일반 대화
1,요새 스터디 그런거 많이 하던데\n스터디\n어떤 거 스터디해\n닌 관심 없니\n내는...,1:요새 스터디 그런거 많이 하던데\n2:스터디\n2:어떤 거 스터디해\n1: 닌 ...,일반 대화,일반 대화
2,오늘 해장국 먹고 옴\n오 무슨 해장국 먹었어요\n홍천 뚝배기 앎 서남 시장에\n오...,1:오늘 해장국 먹고 옴\n2:오 무슨 해장국 먹었어요\n1: 홍천 뚝배기 앎 서남...,일반 대화,일반 대화
3,마블 호크아이 영화 나오나봐\n자기 그 캐릭터 별로잖아\n그렇긴 한데 2세대 ...,1:마블 호크아이 영화 나오나봐\n2:자기 그 캐릭터 별로잖아\n1:그렇긴 한데 2...,일반 대화,일반 대화
4,빕스가까\n확인했나 가\n어어 아직있더라\n죽이제 이번년도 까지인가봐\n...,1:빕스가까\n2:확인했나 가\n1:어어 아직있더라\n1:죽이제 이번년도 까지인가봐...,일반 대화,일반 대화
...,...,...,...,...
17735,준하야 머리가 왜 이렇게 커\n내 머리는 무엇입니까\n밥을 먹으면 머리만 커지나요...,1: 준하야 머리가 왜 이렇게 커\n2: 내 머리는 무엇입니까\n1: 밥을 먹으면...,기타 괴롭힘 대화,공격 대화
17736,당신의 아들 김길준을 데리고 갑니다 그를 구하고 싶다면 그의 계좌로 1억을 보내라...,1: 당신의 아들 김길준을 데리고 갑니다 그를 구하고 싶다면 그의 계좌로 1억을 ...,갈취 대화,공격 대화
17737,선생님 같은 사람들을 보면 정말 이상해요 어떻게 이렇게 살 수 있습니까\n왜 그런...,1: 선생님 같은 사람들을 보면 정말 이상해요 어떻게 이렇게 살 수 있습니까\n1...,직장 내 괴롭힘 대화,공격 대화
17738,여기서 사업을 하고 싶은 사람\n뭐야\n남의 동네에서 장사를 하려면 집세를 내야 한...,1: 여기서 사업을 하고 싶은 사람\n2: 뭐야\n1: 남의 동네에서 장사를 하려면...,갈취 대화,공격 대화


In [ ]:
train_10000.to_csv('/content/drive/MyDrive/AIFFELTHON/TUNiB/experiment_data/train_bt_10000.csv', index=False)